надо было прописать порты и запустить все 3 сервиса
uvicorn recommendations_service:app 
uvicorn features_service:app --port 8010 
uvicorn events_service:app --port 8020 

In [23]:
import pandas as pd

In [24]:
similar=pd.read_parquet("/home/mle-user/mle_projects/mle-project-sprint-4-v001/recs/similar.parquet")

In [25]:
similar.head(3)

,score,track_id_1,track_id_2
1,0.987385,24417,43130
2,0.982825,24417,43127
3,0.981918,24417,55568


In [26]:
pers=pd.read_parquet("/home/mle-user/mle_projects/mle-project-sprint-4-v001/recs/recommendations.parquet")

In [27]:
pers.head(3)

,user_id,track_id,score
0,3,54798445,0.14
1,3,49961817,0.14
2,3,45499814,0.03


это recom_top

top_popular без user_id

In [28]:
pops=pd.read_parquet("/home/mle-user/mle_projects/mle-project-sprint-4-v001/recs/top_popular.parquet")

In [29]:
pops.head(3)

,user_id,track_id,score
0,3,53404,1.00
1,3,178529,0.92
2,3,33311009,0.86


In [30]:
pers.query('user_id==100500100')

,user_id,track_id,score


{'recs': [54798445, 49961817, 45499814]}

Тестирую работу сервисов

новый пользователь - самые популярные

In [31]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 100500100, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

{'recs': [53404, 178529, 33311009]}


существующий пользователь - рассчитанные персональные рекомендации

In [32]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 3, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

{'recs': [54798445, 49961817, 45499814]}


проверяем put и get

сначала убедимся что у пользователя нет events

In [33]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

In [35]:

params = {"user_id": 3}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'events': []}


теперь запишем пользователю в историю прослушанный трек

In [13]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 3, "track_id": 24417}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'result': 'ok'}


проверим что получилось

In [14]:
import requests

params = {"user_id": 3}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'events': [24417]}


online рекомендации

In [15]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

params = {"user_id": 3, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'recs': [43130, 43127, 55568]}


offline рекомендации

In [17]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

params = {"user_id": 3, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'recs': [54798445, 49961817, 45499814]}


итоговые рекомендации

In [18]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

params = {"user_id": 3, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'recs': [43130, 54798445, 43127]}


у пользователя 3 получился микс между онлайн и офлайн рекомендациями